In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

pinky = dj.create_virtual_module('pinky', 'microns_pinky')
schema = dj.schema('microns_pinky')

Connecting celiib@10.28.0.34:3306


In [3]:
#ta3p100.ProofreadLabelOrphan() & "segment_id=648518346341352006"

In [4]:
#ta3p100.CleansedMeshOrphan()

In [5]:
@schema
class CoarseLabelOrphan(dj.Computed):
    definition = """
    # Vertex labels for pinky.ProofreadLabelOrphan did not correctly match the triangle labels, so these are regenerated from the correct triangle labels.
    -> pinky.ProofreadLabelOrphan
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as pinky.ProofreadLabelOrphan
    """
    
    key_source = pinky.ProofreadLabelOrphan & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (pinky.ProofreadLabelOrphan & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (pinky.Decimation35OrphanStitched & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [6]:
start = time.time()
CoarseLabelOrphan.populate(reserve_jobs=True)
print(time.time() - start)

648518346341384946
Segment 648518346341384946 vertex labels regenerated in: 0.5358188152313232 seconds.
648518346341385906
Segment 648518346341385906 vertex labels regenerated in: 0.5467574596405029 seconds.
648518346341387646
Segment 648518346341387646 vertex labels regenerated in: 0.6615636348724365 seconds.
648518346341389643
Segment 648518346341389643 vertex labels regenerated in: 0.604365348815918 seconds.
648518346341392422
Segment 648518346341392422 vertex labels regenerated in: 0.6056551933288574 seconds.
648518346341393799
Segment 648518346341393799 vertex labels regenerated in: 0.7435972690582275 seconds.
648518346341394654
Segment 648518346341394654 vertex labels regenerated in: 0.556786298751831 seconds.
648518346341395278
Segment 648518346341395278 vertex labels regenerated in: 0.632394552230835 seconds.
648518346341396322
Segment 648518346341396322 vertex labels regenerated in: 0.7581644058227539 seconds.
648518346341397381
Segment 648518346341397381 vertex labels regener

DuplicateError: Duplicate entry '3-648518346349513651-0.35-computer_Auto-celiib-2019-06-27 21:26:' for key 'PRIMARY' : To ignore duplicate entries, set skip_duplicates=True in insert.

In [ ]:
(schema.jobs & "table_name='__coarse_label_orphan'").delete()

In [ ]:
ta3p100.CoarseLabelOrphan()

In [ ]:
ta3p100.SegmentExclude()